<a href="https://colab.research.google.com/github/jinsusong/study-nlp-basic/blob/main/Integer_Encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4가지 방법을 통해 단어에 정수를 부여해본다.
1. Python dictionary 자료형 사용하기 
2. Counter() 사용하기
3. NLTK의 FreqDist 사용하기
4. Keras 사용하기 


# dictionary 사용하기 


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
# NLTK는 자연어 처리를 위한 파이썬 패키지
from nltk.tokenize import sent_tokenize  # 영어 문장 토큰화 지원
from nltk.tokenize import word_tokenize  # 토큰의 기준을 단어(word)로 하는 경우, 단어 토큰화(word tokenization)
from nltk.corpus import stopwords 



In [ ]:
raw_text = "A barber is a person. a barber is good person. a barber is huge person. he Knew A Secret! The Secret He Kept is huge secret.\
 Huge secret. His barber kept his word. a barber kept his word. His barber kept his secret. \
 But keeping and keeping such a huge secret to himself was driving the barber crazy. the barber went up a huge mountain."
 

In [ ]:
raw_text

In [ ]:
# 여러 문장을 각 문장으로 토큰화
sentences = sent_tokenize(raw_text)
print(sentences)

- 분리된 문장에 대해서 정제 작업과 정규화 작업을 병행하며, 단어 토큰화를 수행합니다
- 정제 , 정규화 : 같은 의미를 가진 단어는 하나로 만든다
 1. 단어 소문자화로 단어의 개수 통일 
 2. 불용어와 단어 길이가 2이하인 경우에 대해서 단어를 일부 제외 



In [ ]:
vocab = {}
preprocessed_sentences = [] 
stop_words = set(stopwords.words('english'))

for sentence in sentences:
    # 단어 토큰화 
    tokenized_sentence  = word_tokenize(sentence)
    result = []
    #print(tokenized_sentence)

    for word in tokenized_sentence:
        word = word.lower() # 모든 단어를 소문자화하여 단어의 개수를 줄인다. 
        if word not in stop_words: # 단어 토큰화 된 결과에 대해서 불용어를 제거한다. , stop_words 안에 word가 없다면 참 
            if len(word) > 2: # 단어 길이가 2이하인 경우에 대하여 추가로 단어를 제거한다. 
                result.append(word)
                if word not in vocab: 
                    vocab[word] = 0 
                    #print("vocab[",word,"] : " , vocab[word])
                vocab[word] +=1 
                #print(vocab)
    
    preprocessed_sentences.append(result)
print(preprocessed_sentences)






In [ ]:
#  vocab에는 각 단어에 대한 빈도수가 기록
print(vocab)

In [ ]:
# 빈도수가 높은 순서대로 정렬 
vocab_sorted = sorted(vocab.items(), key=lambda x:x[1], reverse = True)
print(vocab_sorted)

In [ ]:
# 높은 빈도수를 가진 단어일수록 낮은 정수를 부여합니다. 정수는 1부터 부여
# 등장 빈도가 낮은 단어는 자연어 처리에서 의미를 가지지 않을 가능성이 높기 때문입니다. 여기서는 빈도수가 1인 단어들은 전부 제외
word_to_index = {}
i = 0 
for (word, frequency) in vocab_sorted :
    if frequency > 1 : # 빈도수가 작은 단어는 제외 
        i = i + 1 
        word_to_index[word] = i 

print(word_to_index)


In [ ]:
# 자연어 처리를 하다보면, 텍스트 데이터에 있는 단어를 모두 사용하기 보다는 빈도수가 가장 높은 n개의 단어만 사용하고 싶은 경우가 많습니다. 
# vocab 상위 5개의 단어만 사용한다고 하면 
vocab_size = 5
# 인덱스가 5 초과인 단어 제거 
words_frequency = [word for word, index in word_to_index.items() if index >= vocab_size +1]

# 해당 단어에 대한 인덱스 정보를 삭제 
for w in words_frequency:
    del word_to_index[w]
print(word_to_index)

- word_to_index에는 빈도수가 높은 상위 5개의 단어만 저장
- word_to_index를 사용하여 단어 토큰화가 된 상태로 저장된 sentences에 있는 각 단어를 정수로 바꾸는 작업

- 예를 들어 sentences에서 첫번째 문장은 ['barber', 'person']이었는데, 이 문장에 대해서는 [1, 5]로 인코딩합니다. 
- 그런데 두번째 문장인 ['barber', 'good', 'person']에는 더 이상 -word_to_index에는 존재하지 않는 단어인 'good'이라는 단어

- OOV 문제 발생 
- 단어 집합에 존재하지 않는 단어들이 생기는 상황을 Out-Of-Vocabulary(단어 집합에 없는 단어) 문제

- word_to_index에 'OOV'란 단어를 새롭게 추가하고, 단어 집합에 없는 단어들은 'OOV'의 인덱스로 인코딩

In [ ]:
word_to_index['OOV'] = len(word_to_index) + 1
print(word_to_index)

In [ ]:
# word_to_index를 사용하여 sentences의 모든 단어들을 맵핑되는 정수로 인코딩
encoded_sentences = []
for sentence in preprocessed_sentences:
    encoded_sentence = []
    for word in sentence:
        try:
            # 단어 집합에 있는 단어라면 해당 단어의 정수를 리턴.
            encoded_sentence.append(word_to_index[word])
        except KeyError:
            # 만약 단어 집합에 없는 단어라면 'OOV'의 정수를 리턴.
            encoded_sentence.append(word_to_index['OOV'])
    encoded_sentences.append(encoded_sentence)
print(encoded_sentences)

좀 더 쉽게 하기 위해서 Counter, FreqDist, enumerate를 사용하거나, 케라스 토크나이저를 사용하는 것을 권장